In [1]:
!pip install -q transformers einops accelerate langchain bitsandbytes xformers

ERROR: Could not install packages due to an OSError: [Errno 13] Permission denied: 'file_structure_representation.cpython-310.pyc'
Consider using the `--user` option or check the permissions.



In [2]:
!nvidia-smi

Fri Jun 16 09:33:25 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 510.47.03    Driver Version: 510.47.03    CUDA Version: 11.6     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   43C    P0    32W / 250W |    255MiB / 16384MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [3]:
from langchain import HuggingFacePipeline
from transformers import AutoTokenizer, pipeline

import torch

model = "tiiuae/falcon-7b-instruct"

tokenizer = AutoTokenizer.from_pretrained(model)

In [4]:
pipeline = pipeline(
    "text-generation", #task
    model=model,
    tokenizer=tokenizer, 
    torch_dtype=torch.bfloat16, 
    trust_remote_code=True,
    device_map="auto",  #manage the memory between CPU and GPU
    max_length=200, 
    do_sample=True, 
    top_k=10, 
    num_return_sequences=1, 
    eos_token_id=tokenizer.eos_token_id
)
    

A new version of the following files was downloaded from https://huggingface.co/tiiuae/falcon-7b-instruct:
- configuration_RW.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.


A new version of the following files was downloaded from https://huggingface.co/tiiuae/falcon-7b-instruct:
- modelling_RW.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Xformers is not installed correctly. If you want to use memory_efficient_attention to accelerate training use the following command to install Xformers
pip install xformers.
The model 'RWForCausalLM' is not supported for text-generation. Supported models are ['BartForCausalLM', 'BertLMHeadModel', 'BertGenerationDecoder', 'BigBirdForCausalLM', 'BigBirdPegasusForCausalLM', 'BioGptForCausalLM', 'BlenderbotForCausalLM', 'BlenderbotSmallForCausalLM', 'BloomForCausalLM', 'CamembertForCausalLM', 'CodeGenForCausalLM', 'CpmAntForCausalLM', 'CTRLLMHeadModel', 'Data2VecTextForCausalLM', 'ElectraForCausalLM', 'ErnieForCausalLM', 'GitForCausalLM', 'GPT2LMHeadModel', 'GPT2LMHeadModel', 'GPTBigCodeForCausalLM', 'GPTNeoForCausalLM', 'GPTNeoXForCausalLM', 'GPTNeoXJapaneseForCausalLM', 'GPTJForCausalLM', 'LlamaForCausalLM', 'MarianForCausalLM', 'MBartForCausalLM', 'MegaForCausalLM', 'MegatronBertForCausalLM', 'MvpForCausalLM', 'OpenLlamaForCausalLM', 'OpenAIGPTLMHeadModel', 'OPTForCausalLM', 'PegasusFor

In [5]:
from langchain import PromptTemplate, LLMChain

In [6]:
llm = HuggingFacePipeline(pipeline=pipeline, model_kwargs={'temperature':0})

In [12]:
%%time
template = """
You are an intelligent chatbot, Help the following question with brilliant answers.
Question: {question}
Answer:
"""
prompt = PromptTemplate(template=template, input_variables=["question"])
llm_chain = LLMChain(prompt=prompt, llm=llm)
question = "Can you explain how large language models can predrict the next word into a sentence?"


print(llm_chain.run(question))

Setting `pad_token_id` to `eos_token_id`:11 for open-end generation.


Large language models use a combination of natural language processing and machine learning algorithms to predict the next word in a sentence. These models analyze patterns in text, learn from examples of sentences, and use them to generate the next word or phrase with a high degree of accuracy. These models use a variety of techniques, such as neural networks, convolutional neural networks, and recurrent neural networks to identify patterns in text which are then used to generate a prediction for the next word or phrase in a sentence. These models are trained on vast amounts of data, which allows them to capture complex language nuances and learn over time. The models also adjust themselves over time by learning from their own mistakes to continually improve their accuracy.
CPU times: user 57.5 s, sys: 0 ns, total: 57.5 s
Wall time: 57.5 s
